<a href="https://colab.research.google.com/github/eel-eel-eel/ric1340/blob/main/ch06_02_exercise_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 固有表現抽出の練習問題

## データ準備・前処理

In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
!unzip chABSA-dataset.zip
!mkdir -p /content/drive/MyDrive/bert/6_2_chABSA
!cp -r chABSA-dataset /content/drive/MyDrive/bert/6_2_chABSA

--2022-08-15 12:30:29--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.4.170
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.4.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K  1.52MB/s    in 0.5s    

2022-08-15 12:30:30 (1.52 MB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

Archive:  chABSA-dataset.zip
   creating: chABSA-dataset/
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflating: 

In [ ]:
label_kinds = []

# make labels (exclude NULL and OOD)
for e in ["market", "company", "business", "product"]:
    for a in ["general", "sales", "profit", "amount", "price", "cost"]:
        label_kinds.append(e + "#" + a)
        if e in ["market"]:
            break;

print(label_kinds)

['market#general', 'company#general', 'company#sales', 'company#profit', 'company#amount', 'company#price', 'company#cost', 'business#general', 'business#sales', 'business#profit', 'business#amount', 'business#price', 'business#cost', 'product#general', 'product#sales', 'product#profit', 'product#amount', 'product#price', 'product#cost']


In [ ]:
import json
import glob
import pandas as pd
from collections import Counter
import re


dataset = []
labels = []

for f in glob.glob("/content/drive/MyDrive/bert/6_2_chABSA/chABSA-dataset/e*_ann.json"):
    with open(f, encoding="utf-8") as j:
        d = json.load(j)
        for s in d["sentences"]:
            os = []
            cs = []
            ws = []
            from_to = []
            temp_dict = []
            for o in s["opinions"]:
                category = o["category"]
                word = o["target"]

                if category in label_kinds and category not in cs:
                  cs.append(category)
                  category = re.sub("#\D*", "", o["category"])
                  temp_dict2 = dict(name=word, span=[o["from"],o["to"]],type=category)
                  temp_dict.append(temp_dict2)

            if len(cs) > 0:
                dataset.append(
                    {"sentence": s["sentence"],
                     "entities": temp_dict}
                )


dataset = pd.DataFrame(dataset)
print(dataset.head(5))

                                            sentence  \
0  資源・金属素材関連では、世界的な政治の混乱などが実体経済へ波及し、資源需要が全般的に低迷した...   
1  産機・建機関連では、企業収益や設備投資にやや改善傾向が見られたことから、民間企業向け各種ポン...   
2  環境設備関連では、主力商品、関連商品とも堅調に推移し、水砕関連もほぼ計画のとおりとなったこと...   
3  プラント・設備工事関連では、大口受注工事の完工などから、関連部門の売上高は45億１百万円とな...   
4  化成品関連では、自動車関連及び一部電線業界が堅調に推移したものの、原油価格の低迷から販売価格...   

                                            entities  
0  [{'name': '資源・金属素材関連', 'span': [0, 9], 'type':...  
1  [{'name': '産機・建機関連', 'span': [0, 7], 'type': '...  
2  [{'name': '環境設備関連', 'span': [0, 6], 'type': 'b...  
3  [{'name': 'プラント・設備工事関連', 'span': [0, 11], 'typ...  
4  [{'name': '化成品関連', 'span': [0, 5], 'type': 'bu...  


In [ ]:
dataset.head

<bound method NDFrame.head of                                                sentence  \
0     資源・金属素材関連では、世界的な政治の混乱などが実体経済へ波及し、資源需要が全般的に低迷した...   
1     産機・建機関連では、企業収益や設備投資にやや改善傾向が見られたことから、民間企業向け各種ポン...   
2     環境設備関連では、主力商品、関連商品とも堅調に推移し、水砕関連もほぼ計画のとおりとなったこと...   
3     プラント・設備工事関連では、大口受注工事の完工などから、関連部門の売上高は45億１百万円とな...   
4     化成品関連では、自動車関連及び一部電線業界が堅調に推移したものの、原油価格の低迷から販売価格...   
...                                                 ...   
1339  同事業の販売状況につきましては、自動車部品向けや、中国販売子会社での販売増により、販売数量は...   
1340  電子部品向け金属粉の販売状況につきましては、スマートフォン関連需要に加え、軟磁性材向けの販売...   
1341  粉末冶金向け金属粉の販売状況につきましては、自動車部品向けが好調な一方、熊本地震等の影響によ...   
1342  セグメント利益は、電子部品向け金属粉の販売好調により、同186百万円増益（32.6％増益）の...   
1343  不動産賃貸事業の当連結会計年度の売上高は23百万円（前年度比18.5%増収）、セグメント利益...   

                                               entities  
0     [{'name': '資源・金属素材関連', 'span': [0, 9], 'type':...  
1     [{'name': '産機・建機関連', 'span': [0, 7], 'type': '...  
2     [{'name': '環境設備関連', 'span': [0, 6], 'type': 'b...  
3     [{'name': 'プラント・設備工事関連'

In [ ]:
max_len = 0

for i in range(dataset.shape[0]):
  if max_len < len(dataset.loc[i, "sentence"]):
    max_len = len(dataset.loc[i, "sentence"])
print("最大文字数：",max_len)

最大文字数： 334


In [ ]:
!pip install transformers[ja]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.1 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 568 kB 64.6 MB/s 
     |████████████████████████████████| 47.4 MB 133 kB/s 
     |████████████████████████████████| 13.4 MB 48.9 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=37c5b96a112f9ace130606bc2ff97e9fc70aa8f5d0eed603cc3cbc3e5a5539f0
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=ca09c30875008efccae1d0cda6a49105798697741deebff7c4377ef175990a20
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e

In [ ]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

In [ ]:
list_text = [dataset.loc[i,"sentence"] for i in range(dataset.shape[0])]
list_tokens = [tokenizer.tokenize(text) for text in list_text]
list_entities = [dataset.loc[i,"entities"] for i in range(dataset.shape[0])]

In [ ]:
# cl-tohoku/bert-base-japanese-whole-word-maskingのモデルは最大512トークンまで対応しているが、
# 学習時のGPUメモリ消費を抑えるため256としている
n_token = 256

# encode後のトークンは、特殊トークン（{CLS], {SEP]など）や特殊文字（##）が挿入されることに注意
# spanで表されているラベルと文字数ベースで位置ずれを起こすため、BIO形式に変換する際に補正する必要がある
list_text_id =  [tokenizer.encode(text, truncation=True, padding='max_length', max_length=n_token) for text in list_text]
list_tokens = [tokenizer.convert_ids_to_tokens(encode) for encode in list_text_id]

In [ ]:
def is_in_span(idx, span):
  return span[0] <= idx and idx < span[1]

In [ ]:
from collections import defaultdict
list_bio =[]

label2id = defaultdict(lambda :len(label2id))
_ = label2id["O"]

for text, tokens, entities in zip(list_text, list_tokens, list_entities):
  bio = ["O"] * len(tokens)

  for entity in entities:
    cnt = 0
    begin_flg = True
    label = entity["type"]

    for i, tok in enumerate(tokens):
      if tok == "[CLS]" or tok == "[SEP]" or tok == "[PAD]":
        continue
      elif is_in_span(cnt, entity["span"]):
        if begin_flg:
          bio[i] = f"B-{label}"
          begin_flg = False
          _ = label2id[f"B-{label}"]
          _ = label2id[f"I-{label}"]
        else:
          bio[i] = f"I-{label}"

      cnt += len(tok.replace("##", ""))

  list_bio.append(bio)

id2label = {v:k for k,v in label2id.items()}

In [ ]:
list_bio_id = [[label2id[label] for label in bio] for bio in list_bio]

In [ ]:
# BIO形式に変換できていることの確認
for dec, bio, _ in zip(list_tokens[0], list_bio[0], range(100)):
  print(f"{dec}\t{bio}")

[CLS]	O
資源	B-business
・	I-business
金属	I-business
素材	I-business
関連	I-business
で	O
は	O
、	O
世界	O
的	O
な	O
政治	O
の	O
混乱	O
など	O
が	O
実体	O
経済	O
へ	O
波及	O
し	O
、	O
資源	O
需要	O
が	O
全般	O
的	O
に	O
低迷	O
し	O
た	O
こと	O
から	O
、	O
関連	O
部門	O
の	O
売上	O
高	O
は	O
93	O
億	O
3	O
百	O
万	O
円	O
と	O
なり	O
、	O
前年	O
同期	O
と	O
比べ	O
13	O
億	O
46	O
百	O
万	O
円	O
(	O
##△	O
12	O
.	O
6	O
%)	O
の	O
減	O
##収	O
と	O
なり	O
まし	O
た	O
[SEP]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O


In [ ]:
len(list_bio)

1344

In [ ]:
import torch

# 使用デバイスにGPUを設定
# 以下のような出力が出ていれば正常に設定ができている
# device(type='cuda', index=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from torch.utils.data import Dataset

class NERDataset(Dataset):
  def __init__(self, texts_id, bios_id, is_test=False):
    self.texts_id = texts_id
    self.bios_id = bios_id
    self.is_test = is_test

  def __getitem__(self, idx):
    data = {'input_ids': torch.tensor(self.texts_id[idx], device=device)}
    if not self.is_test:
      data['label'] = torch.tensor(self.bios_id[idx], device=device)
    return data

  def __len__(self):
    return len(self.bios_id)

データ分割

In [ ]:
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

In [ ]:
from sklearn.model_selection import train_test_split


list_text_id_train, list_text_id_test, list_bio_id_train, list_bio_id_test = \
    train_test_split(list_text_id, list_bio_id, test_size=test_ratio, random_state=42)
list_text_id_train, list_text_id_valid, list_bio_id_train, list_bio_id_valid = \
    train_test_split(list_text_id_train, list_bio_id_train, test_size=val_ratio/(test_ratio+train_ratio), random_state=42)



データを.txt形式で保存しておく

In [ ]:
!mkdir -p /content/drive/MyDrive/bert/6_2_chABSA/dataset/

import pickle
# データの保存
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_train.txt', 'wb')
pickle.dump(list_text_id_train, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_valid.txt', 'wb')
pickle.dump(list_text_id_valid, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_test.txt', 'wb')
pickle.dump(list_text_id_test, f)

f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_train.txt', 'wb')
pickle.dump(list_bio_id_train, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_valid.txt', 'wb')
pickle.dump(list_bio_id_valid, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_test.txt', 'wb')
pickle.dump(list_bio_id_test, f)


# データの読み込み
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_train.txt', 'rb')
# list_text_id_train = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_valid.txt', 'rb')
# list_text_id_valid = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_test.txt', 'rb')
# list_text_id_test = pickle.load(f)

# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_train.txt', 'rb')
# list_bio_id_train = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_valid.txt', 'rb')
# list_bio_id_valid = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_test.txt', 'rb')
# list_bio_id_test = pickle.load(f)

In [ ]:
ds_train = NERDataset(list_text_id_train, list_bio_id_train)
ds_valid = NERDataset(list_text_id_valid, list_bio_id_valid)
ds_test = NERDataset(list_text_id_test, list_bio_id_test, is_test=True)

print("学習データ数：", len(ds_train))
print("検証データ数：", len(ds_valid))
print("テストデータ数：", len(ds_test))

学習データ数： 806
検証データ数： 269
テストデータ数： 269


## 固有表現抽出モデルの作成

## 解答1

In [ ]:
# Q. BertForTokenClassificationを使って、事前学習済みモデルを読み込みましょう

from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
    pretrained_model_name_or_path='cl-tohoku/bert-base-japanese-whole-word-masking',
    id2label=id2label,
    label2id=label2id)

model.to(device)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
!mkdir -p /content/drive/MyDrive/bert/6_2_chABSA/results
from transformers import Trainer, TrainingArguments
import torch


## 解答2

In [ ]:
batch_size = 8

# Q. training_configとtrainerを定義しましょう

training_config = TrainingArguments(
  output_dir='/content/drive/MyDrive/bert/6_2_chABSA/results',
  num_train_epochs=20,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  warmup_steps=500,
  weight_decay=0.2,
  save_steps=500,
  do_eval=True,
  eval_steps=100
)

trainer = Trainer(
    model=model,
    args=training_config,
    tokenizer=tokenizer,
    train_dataset=ds_train,
    eval_dataset=ds_valid
)

## 解答3

In [ ]:
# Q. trainerを使って、学習しましょう

trainer.train() # この行に学習を行うコードを実装しましょう

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 806
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2020


Step,Training Loss
500,0.121100
1000,0.007600
1500,0.001500
2000,0.000400


Saving model checkpoint to /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-500
Configuration saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-1000
Configuration saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/bert/6_2_chABSA/results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/bert/6_2_chABSA/resul

TrainOutput(global_step=2020, training_loss=0.032334402688239505, metrics={'train_runtime': 801.3531, 'train_samples_per_second': 20.116, 'train_steps_per_second': 2.521, 'total_flos': 2106185144094720.0, 'train_loss': 0.032334402688239505, 'epoch': 20.0})

## テストデータの推論

In [ ]:
# 固有表現抽出の評価用ライブラリをインストール
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 695 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=60a54009d6d46ba5e1489e705b02d804ca86fe8828e2920b9ec014a242a417ca
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


## 解答4

In [ ]:
# Q. trainerを使って、推論結果をresult変数としましょう

result = trainer.predict(ds_test) # この行にテストデータで推論を行うコードを実装しましょう

***** Running Prediction *****
  Num examples = 269
  Batch size = 8


In [ ]:
import numpy as np

trues =  np.vectorize(lambda x:id2label[x])(ds_test.bios_id).tolist()

preds_id = np.argmax(result.predictions, axis=2)
preds = np.vectorize(lambda x:id2label[x])(preds_id).tolist()

In [ ]:
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

print(f1_score(trues, preds))

print(classification_report(trues, preds))

0.6149341142020498
              precision    recall  f1-score   support

    business       0.66      0.74      0.70       148
     company       0.12      0.25      0.17         8
      market       0.44      0.57      0.50        21
     product       0.56      0.60      0.58       145

   micro avg       0.58      0.65      0.61       322
   macro avg       0.45      0.54      0.49       322
weighted avg       0.59      0.65      0.62       322

